In [2]:
# Load libraries

import os
import json
import sys
from datetime import datetime

In [3]:
cd ..

/Users/linafaik/Documents/projects/agentic-investor-brief


In [5]:
from agent_investor_brief.config import setup_environment
from agent_investor_brief.agents.investor_agent import InvestorAgent
from agent_investor_brief.tools.industry_research import IndustryResearchTool
from agent_investor_brief.tools.financial_data import FinancialDataTool
from agent_investor_brief.tools.analysis_builder import AnalysisBuilderTool

⚠️  Warning: OPENAI_API_KEY not set. Some features may not work.
✅ Environment setup completed
📊 MLflow tracking: sqlite:///mlflow.db
🤖 Default model: gpt-4o-mini
📁 Output directory: outputs


KeyboardInterrupt: 

In [ ]:
#!/usr/bin/env python3
"""
Agent Investor Brief - Simple Demo Script

This script demonstrates the MLflow + LangChain powered agent that creates 
comprehensive investment briefs for any publicly traded company.
"""



def setup_demo():
    """Setup the demo environment"""
    print("🚀 Agent Investor Brief - Demo")
    print("=" * 50)
    
    # Add src to path for development
    src_path = os.path.join(os.path.dirname(__file__), '..', 'src')
    if src_path not in sys.path:
        sys.path.insert(0, src_path)
    
    try:
        from agent_investor_brief.config import setup_environment
        from agent_investor_brief.agents.investor_agent import InvestorAgent
        from agent_investor_brief.tools.industry_research import IndustryResearchTool
        from agent_investor_brief.tools.financial_data import FinancialDataTool
        from agent_investor_brief.tools.analysis_builder import AnalysisBuilderTool
        
        print("✅ Imports successful!")
        return True
        
    except ImportError as e:
        print(f"❌ Import failed: {e}")
        print("💡 Try: pip install -e . or uv pip install -e .")
        return False

def check_api_key():
    """Check if OpenAI API key is configured"""
    api_key = os.getenv("OPENAI_API_KEY")
    if not api_key:
        print("\n⚠️  OpenAI API Key Not Found!")
        print("To run with real LLM:")
        print("1. Set environment: export OPENAI_API_KEY='your-key-here'")
        print("2. Or create .env file with: OPENAI_API_KEY=your-key-here")
        print("\n🎭 Running in simulated mode instead...")
        return False
    
    print("✅ OpenAI API key configured")
    return True

def demo_individual_tools():
    """Demo each tool individually"""
    print("\n" + "=" * 50)
    print("🧪 DEMO 1: Individual Tools")
    print("=" * 50)
    
    from agent_investor_brief.tools.industry_research import IndustryResearchTool
    from agent_investor_brief.tools.financial_data import FinancialDataTool
    from agent_investor_brief.tools.analysis_builder import AnalysisBuilderTool
    
    # Test Industry Research
    print("\n📊 Testing Industry Research Tool...")
    research_tool = IndustryResearchTool()
    industry_result = research_tool._run("Apple")
    industry_data = json.loads(industry_result)
    
    print(f"✅ Research completed for: {industry_data.get('query', 'N/A')}")
    print(f"📝 Summary preview: {industry_data.get('research_summary', '')[:150]}...")
    
    # Test Financial Data
    print("\n💰 Testing Financial Data Tool...")
    financial_tool = FinancialDataTool()
    
    try:
        financial_result = financial_tool._run("AAPL")
        financial_data = json.loads(financial_result)
        
        if "error" not in financial_result:
            company_info = financial_data.get('detailed_data', {}).get('company_info', {})
            price_data = financial_data.get('detailed_data', {}).get('price_data', {})
            
            print(f"✅ Data fetched for: {company_info.get('company_name', 'Unknown')}")
            print(f"💵 Current price: ${price_data.get('current_price', 0):.2f}")
            print(f"🏢 Sector: {company_info.get('sector', 'Unknown')}")
        else:
            print("⚠️  Using simulated financial data")
            financial_data = create_simulated_financial_data()
            
    except Exception as e:
        print(f"⚠️  Financial data error: {e}")
        print("📊 Using simulated data instead")
        financial_data = create_simulated_financial_data()
    
    # Test Analysis Builder
    print("\n🔍 Testing Analysis Builder Tool...")
    analysis_tool = AnalysisBuilderTool()
    
    analysis_input = json.dumps({
        "financial_data": financial_data,
        "industry_research": industry_data,
        "ticker": "AAPL"
    })
    
    analysis_result = analysis_tool._run(analysis_input)
    analysis_data = json.loads(analysis_result)
    
    if "investor_brief" in analysis_data:
        print("✅ Investment analysis completed")
        recommendation = analysis_data.get("detailed_analysis", {}).get("investment_recommendation", {})
        print(f"📈 Recommendation: {recommendation.get('recommendation', 'N/A')}")
        print(f"🎯 Price target: ${recommendation.get('price_target', 0):.2f}")
    
    return financial_data, industry_data, analysis_data

def demo_full_agent(has_api_key):
    """Demo the complete agent workflow"""
    print("\n" + "=" * 50)
    print("🤖 DEMO 2: Complete Agent Workflow")
    print("=" * 50)
    
    if has_api_key:
        try:
            from agent_investor_brief.agents.investor_agent import InvestorAgent
            
            print("🚀 Initializing Investor Agent...")
            agent = InvestorAgent()
            
            # Create investment brief
            ticker = "AAPL"
            print(f"\n📋 Creating investment brief for {ticker}...")
            
            result = agent.create_investment_brief(ticker)
            
            if "error" not in result:
                print("✅ Investment brief created successfully!")
                
                brief = result.get("investment_brief", "")
                print(f"📄 Brief length: {len(brief.split())} words")
                
                print(f"\n📋 Brief Preview:")
                print("-" * 40)
                print(brief[:400] + "...")
                print("-" * 40)
                
                # Test conversational Q&A
                print(f"\n💬 Testing conversational Q&A...")
                question = f"What are the main risks for {ticker}?"
                print(f"❓ Question: {question}")
                
                response = agent.ask_question(question)
                print(f"🤖 Response: {response[:200]}...")
                
                return result
            else:
                print(f"❌ Error: {result.get('error', 'Unknown error')}")
                
        except Exception as e:
            print(f"❌ Agent demo failed: {e}")
            print("📊 Falling back to simulated mode")
            has_api_key = False
    
    if not has_api_key:
        print("🎭 Simulated Agent Demo:")
        simulated_brief = create_simulated_brief("AAPL")
        print("✅ Simulated brief created!")
        print(f"📄 Brief length: {len(simulated_brief.split())} words")
        print(f"\n📋 Brief Preview:")
        print("-" * 40)
        print(simulated_brief[:400] + "...")
        
        return {"investment_brief": simulated_brief, "company_ticker": "AAPL"}

def demo_mlflow_integration():
    """Demo MLflow tracking capabilities"""
    print("\n" + "=" * 50)
    print("📊 DEMO 3: MLflow Integration")
    print("=" * 50)
    
    try:
        import mlflow
        from agent_investor_brief.config import settings
        
        print(f"📈 MLflow Experiment: {settings.mlflow_experiment_name}")
        print(f"🔗 Tracking URI: {settings.mlflow_tracking_uri}")
        
        # Check if experiment exists
        try:
            experiment = mlflow.get_experiment_by_name(settings.mlflow_experiment_name)
            if experiment:
                print(f"📊 Experiment ID: {experiment.experiment_id}")
                
                # Get recent runs
                runs = mlflow.search_runs(experiment_ids=[experiment.experiment_id], max_results=3)
                if not runs.empty:
                    print(f"\n📋 Recent runs ({len(runs)} found):")
                    for idx, run in runs.iterrows():
                        run_name = run.get('tags.mlflow.runName', 'Unnamed')[:40]
                        status = run.get('status', 'Unknown')
                        print(f"  • {run_name}... - {status}")
                else:
                    print("📋 No runs found yet - create your first brief!")
            else:
                print("📊 Experiment will be created on first run")
                
        except Exception as e:
            print(f"⚠️  Experiment info unavailable: {e}")
        
        print(f"\n🌐 To view MLflow UI:")
        print(f"1. Run: mlflow ui")
        print(f"2. Open: http://localhost:5000")
        print(f"3. Navigate to experiment: {settings.mlflow_experiment_name}")
        print(f"4. Click 'Traces' tab for detailed execution flows")
        
    except ImportError:
        print("❌ MLflow not available")
    except Exception as e:
        print(f"⚠️  MLflow demo error: {e}")

def create_simulated_financial_data():
    """Create simulated financial data for demo"""
    return {
        "detailed_data": {
            "company_info": {
                "company_name": "Apple Inc.",
                "sector": "Technology",
                "market_cap": 3000000000000
            },
            "price_data": {
                "current_price": 185.50,
                "day_change": 1.2,
                "returns": {"ytd": 8.5}
            },
            "key_metrics": {
                "valuation_metrics": {"pe_ratio": 29.5, "pb_ratio": 45.2},
                "profitability_metrics": {"roe_pct": 160.0, "profit_margin_pct": 26.3},
                "financial_health": {"debt_to_equity": 0.15, "current_ratio": 1.8}
            }
        },
        "data_quality": {"completeness": "excellent"}
    }

def create_simulated_brief(ticker):
    """Create simulated investment brief"""
    return f"""# Investment Brief: {ticker}

**Recommendation**: Buy
**Price Target**: $200.00 | **Upside**: +15.2%
**Time Horizon**: 12 months

## Investment Thesis
{ticker} presents a compelling investment opportunity driven by strong fundamentals, 
innovative product pipeline, and expanding market opportunities. The company's 
solid balance sheet and cash generation capabilities support continued growth.

## Key Investment Strengths
• Market leadership in core segments
• Strong brand recognition and customer loyalty  
• Robust financial performance with consistent profitability
• Innovation-driven growth strategy

## Key Concerns
• Valuation at premium levels
• Competitive pressure in key markets
• Regulatory environment uncertainty

## Financial Quality Assessment
**Overall Grade**: A-
**Quality Tier**: High Quality

## Risk Profile  
**Risk Grade**: Medium
**Overall Risk Score**: 45/100

## Scenario Analysis
**Expected Return**: +12.5%
• Bull Case: +25.0% (25% probability)
• Base Case: +15.0% (50% probability)
• Bear Case: -5.0% (25% probability)

---
*This analysis is for informational purposes only and should not be considered as investment advice.*
"""

def demo_batch_analysis():
    """Demo batch analysis of multiple companies"""
    print("\n" + "=" * 50)
    print("📊 DEMO 4: Batch Analysis")
    print("=" * 50)
    
    tickers = ["AAPL", "TSLA", "MSFT"]
    print(f"📈 Analyzing portfolio: {', '.join(tickers)}")
    
    results = {}
    for ticker in tickers:
        print(f"\n🔄 Processing {ticker}...")
        
        # Simulated results for demo
        brief = create_simulated_brief(ticker)
        results[ticker] = {
            "ticker": ticker,
            "recommendation": "Buy" if "Buy" in brief else "Hold",
            "brief_length": len(brief.split()),
            "analysis_date": datetime.now().strftime("%Y-%m-%d")
        }
        print(f"✅ {ticker} analysis completed")
    
    # Show comparison
    print(f"\n📋 Portfolio Analysis Summary:")
    print("-" * 60)
    for ticker, data in results.items():
        print(f"{ticker:6} | {data['recommendation']:8} | {data['brief_length']:4} words | {data['analysis_date']}")
    print("-" * 60)
    
    return results

def main():
    """Main demo function"""
    # Setup
    if not setup_demo():
        return
    
    # Check API configuration
    has_api_key = check_api_key()
    
    try:
        from agent_investor_brief.config import setup_environment
        settings = setup_environment()
        print(f"🔧 Environment configured successfully")
    except Exception as e:
        print(f"⚠️  Environment setup warning: {e}")
    
    # Run demos
    try:
        # Demo 1: Individual tools
        financial_data, industry_data, analysis_data = demo_individual_tools()
        
        # Demo 2: Complete agent workflow  
        agent_result = demo_full_agent(has_api_key)
        
        # Demo 3: MLflow integration
        demo_mlflow_integration()
        
        # Demo 4: Batch analysis
        batch_results = demo_batch_analysis()
        
        # Summary
        print("\n" + "=" * 50)
        print("🎉 DEMO COMPLETED SUCCESSFULLY!")
        print("=" * 50)
        
        print("\n✅ Demonstrations completed:")
        print("1. 🔧 Individual tool testing")
        print("2. 🤖 Complete agent workflow")  
        print("3. 📊 MLflow experiment tracking")
        print("4. 📈 Batch portfolio analysis")
        
        print("\n🚀 Key Features Showcased:")
        print("• MLflow one-line tracing with autolog()")
        print("• Yahoo Finance data integration")
        print("• Industry research and analysis")
        print("• Investment recommendations with risk assessment")
        print("• Conversational Q&A capabilities")
        print("• Professional brief generation")
        
        print("\n📖 Next Steps:")
        print("1. Set OPENAI_API_KEY for full functionality")
        print("2. Run 'mlflow ui' to explore traces")
        print("3. Customize tools for your use case")
        print("4. Add more companies to analyze")
        
        if not has_api_key:
            print("\n💡 To enable full functionality:")
            print("export OPENAI_API_KEY='your-key-here'")
            print("python demo.py")
        
        print("\n🎯 Happy Investing! 📈")
        
    except Exception as e:
        print(f"\n❌ Demo error: {e}")
        print("🔧 Check your setup and try again")

if __name__ == "__main__":
    main()